# Comparison of fibroblast populations (review after JID)

In this notebook we are going to extract and replicate the main populations from diffrent papers where fibroblast populations are described, and find similarities and differences. The premise of this analysis is that many of the populations described in different papers seem not to match, or to be transcriptomically different, but in reality they are quite similar; that is, the main types of populations are indeed shared by the different papers, which should come as no surprise.

**After the publication in JID we will include the following papers, as confirmatory results**
* Kim et al. 2020

The data from He et al was reanalyzed from fastq files with healthy donor due to the strong batch effects (the samples were already normalized / log transformed, which limits the scope of the downstream processing), and some important genes such as WIF1 were not appearing. 

## imports

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import ray
import subprocess
import time
import scvelo as scv
import gc

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
dict_rep = {'CCN5': 'WISP2', 'ECRG4': 'C2orf40'}

In [ ]:
mpl.rcParams['figure.dpi'] = 150

**IMPORTANT: I am running this analysis in a computer with ~500 GB of RAM. I will load many datasets at once, which might be too much for some computers. I took this decision conciously, to have as much info available at any time as possible. If you cannot run all the analysis at once, you can run it by parts.**

## data extraction and processing

In [ ]:
data_dir = os.getcwd()
print(data_dir)

### Kim et al. 2020

#### Raw data and metadata extraction

In [ ]:
kim_dir = data_dir + '/Kim_2020'
os.makedirs(kim_dir, exist_ok=True)
os.makedirs(kim_dir + '/injury', exist_ok=True)

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307706 --gzip --split-files

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307706 --gzip --split-files

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307708 --gzip --split-files

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307709 --gzip --split-files

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307710 --gzip --split-files

In [ ]:
!cd {kim_dir} && fastq-dump SRR9307711 --gzip --split-files

In [ ]:
!cd {kim_dir}/injury && fastq-dump SRR9307698 --gzip --split-files

In [ ]:
df = pd.DataFrame({'name': ['Kim_2020_HC1', 'Kim_2020_HC2', 'Kim_2020_HC3', 
                            'Kim_2020_HC4', 'Kim_2020_HC5', 'Kim_2020_HC6', 
                            'Kim_2020_inj'], 'technology': ['10xv2'] * 7, 
                   'targetnumcells': [1000] * 7})
df.to_csv(kim_dir + '/metadata.tab', sep='\t', index=None)

In [ ]:
!mv {kim_dir}/injury/SRR9307698_2.fastq.gz {kim_dir}/Kim_2020_inj_L001_R1_001.fastq.gz 
!mv {kim_dir}/injury/SRR9307698_3.fastq.gz {kim_dir}/Kim_2020_inj_L001_R2_001.fastq.gz 

!mv {kim_dir}/SRR9307706_2.fastq.gz {kim_dir}/Kim_2020_HC1_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307706_3.fastq.gz {kim_dir}/Kim_2020_HC1_L001_R2_001.fastq.gz 
!mv {kim_dir}/SRR9307707_2.fastq.gz {kim_dir}/Kim_2020_HC2_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307707_3.fastq.gz {kim_dir}/Kim_2020_HC2_L001_R2_001.fastq.gz 
!mv {kim_dir}/SRR9307708_2.fastq.gz {kim_dir}/Kim_2020_HC3_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307708_3.fastq.gz {kim_dir}/Kim_2020_HC3_L001_R2_001.fastq.gz 
!mv {kim_dir}/SRR9307709_2.fastq.gz {kim_dir}/Kim_2020_HC4_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307709_3.fastq.gz {kim_dir}/Kim_2020_HC4_L001_R2_001.fastq.gz 
!mv {kim_dir}/SRR9307710_2.fastq.gz {kim_dir}/Kim_2020_HC5_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307710_3.fastq.gz {kim_dir}/Kim_2020_HC5_L001_R2_001.fastq.gz 
!mv {kim_dir}/SRR9307711_2.fastq.gz {kim_dir}/Kim_2020_HC6_L001_R1_001.fastq.gz 
!mv {kim_dir}/SRR9307711_3.fastq.gz {kim_dir}/Kim_2020_HC6_L001_R2_001.fastq.gz 

In [ ]:
!rm -rf {kim_dir}/*_1.fastq.gz

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC1.loom Kim_2020_HC1 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC1_L001_R1_001.fastq.gz Kim_2020_HC1_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC2.loom Kim_2020_HC2 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC2_L001_R1_001.fastq.gz Kim_2020_HC2_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC3.loom Kim_2020_HC3 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC3_L001_R1_001.fastq.gz Kim_2020_HC3_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC4.loom Kim_2020_HC4 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC4_L001_R1_001.fastq.gz Kim_2020_HC4_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC5.loom Kim_2020_HC5 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC5_L001_R1_001.fastq.gz Kim_2020_HC5_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_HC6.loom Kim_2020_HC6 /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_HC6_L001_R1_001.fastq.gz Kim_2020_HC6_L001_R2_001.fastq.gz 

In [ ]:
!cd {kim_dir} && loompy fromfq Kim_2020_inj.loom Kim_2020_inj /media/seth/SETH_DATA/SETH_Alex/Programs/human_GRCh38_gencode.v31.600 metadata.tab \
Kim_2020_inj_L001_R1_001.fastq.gz Kim_2020_inj_L001_R2_001.fastq.gz

#### Adata creation and metadata gathering

In [ ]:
adata_kim_HC1 = sc.read_loom(kim_dir + '/Kim_2020_HC1.loom').var_names_make_unique()
adata_kim_HC2 = sc.read_loom(kim_dir + '/Kim_2020_HC2.loom').var_names_make_unique()
adata_kim_HC3 = sc.read_loom(kim_dir + '/Kim_2020_HC3.loom').var_names_make_unique()
adata_kim_HC4 = sc.read_loom(kim_dir + '/Kim_2020_HC4.loom').var_names_make_unique()
adata_kim_HC5 = sc.read_loom(kim_dir + '/Kim_2020_HC5.loom').var_names_make_unique()
adata_kim_HC6 = sc.read_loom(kim_dir + '/Kim_2020_HC6.loom').var_names_make_unique()

In [ ]:
adata_kim = sc.AnnData.concatenate(adata_kim_HC1, adata_kim_HC2, adata_kim_HC3, 
                                  adata_kim_HC4, adata_kim_HC5, adata_kim_HC6)

In [ ]:
adata_kim.var_names = [dict_rep[i] if i in dict_rep else i for i in adata_kim.var_names ]

In [ ]:
sc.pp.filter_genes(adata_kim, min_counts=1)

In [ ]:
adata_kim.X = np.array(adata_kim.X.todense())

In [ ]:
# Basic QC filtering
adata_kim.var['mt'] = adata_kim.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_kim, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(adata_kim, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(adata_kim, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_kim, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata_kim = adata_kim[((adata_kim.obs.n_genes_by_counts < 4500) & 
                                    (adata_kim.obs.n_genes_by_counts > 400)).values, :]
adata_kim = adata_kim[adata_kim.obs.pct_counts_mt < 25, :]

In [ ]:
sc.pp.filter_genes(adata_kim, min_counts=1)
sc.pp.log1p(adata_kim)
sc.pp.normalize_total(adata_kim)

In [ ]:
sc.pp.pca(adata_kim, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_kim, metric='angular', batch_key='sample')
tk.tl.triku(adata_kim, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_kim, min_dist=0.1, random_state=seed)
sc.tl.leiden(adata_kim, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_kim, color=['leiden', 'sample'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_kim, color=['leiden', 'LUM', 'PDGFRA', 'COL1A1', 'DCN'], 
           legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
adata_kim_fb = adata_kim[adata_kim.obs['leiden'].isin(['0', '1', '3', '8', '9', '31'])]

### Gaydosik et al. 2020

#### Raw data and metadata extraction

In [ ]:
gaydosik_dir = data_dir + '/gaydosik_2020'
os.makedirs(gaydosik_dir, exist_ok=True)

#### Adata creation and metadata gathering

In [ ]:
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679033/suppl/GSM3679033%5FLabeled%5FSC67%5F050517%5FSK%5FMF2%5FGRCh38raw%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679034/suppl/GSM3679034%5FLabeled%5FSC82%5F060617%5FSK%5FMF5%5FGRCh38raw%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679035/suppl/GSM3679035%5FSC157dataframe%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679036/suppl/GSM3679036%5FSC158dataframe%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679037/suppl/GSM3679037%5FSC205dataframe%2Ecsv%2Egz

!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679038/suppl/GSM3679038%5FLabeled%5FSC50%5F011917%5FSK%5FNOR%5FGRCh38raw%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679039/suppl/GSM3679039%5FLabeled%5FSC68%5F051517%5FSK%5FNOR%5FGRCh38raw%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679040/suppl/GSM3679040%5FLabeled%5FSC124%5F080317%5FSK%5FNOR%5FGRCh38raw%2Ecsv%2Egz
!cd {gaydosik_dir} && wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3679nnn/GSM3679041/suppl/GSM3679041%5FLabeled%5FSC125%5F080317%5FSK%5FNOR%5FGRCh38raw%2Ecsv%2Egz

In [ ]:
!cd {gaydosik_dir} &&  gunzip *.gz

In [ ]:
adata_CTCL2 = sc.read_csv(gaydosik_dir + '/GSM3679033_Labeled_SC67_050517_SK_MF2_GRCh38raw.csv').transpose()
adata_CTCL5 = sc.read_csv(gaydosik_dir + '/GSM3679034_Labeled_SC82_060617_SK_MF5_GRCh38raw.csv').transpose()
adata_CTCL6 = sc.read_csv(gaydosik_dir + '/GSM3679035_SC157dataframe.csv').transpose()
adata_CTCL8 = sc.read_csv(gaydosik_dir + '/GSM3679036_SC158dataframe.csv').transpose()
adata_CTCL12 = sc.read_csv(gaydosik_dir + '/GSM3679037_SC205dataframe.csv').transpose()

adata_HC1 = sc.read_csv(gaydosik_dir + '/GSM3679038_Labeled_SC50_011917_SK_NOR_GRCh38raw.csv').transpose()
adata_HC2 = sc.read_csv(gaydosik_dir + '/GSM3679039_Labeled_SC68_051517_SK_NOR_GRCh38raw.csv').transpose()
adata_HC3 = sc.read_csv(gaydosik_dir + '/GSM3679040_Labeled_SC124_080317_SK_NOR_GRCh38raw.csv').transpose()
adata_HC4 = sc.read_csv(gaydosik_dir + '/GSM3679041_Labeled_SC125_080317_SK_NOR_GRCh38raw.csv').transpose()

In [ ]:
adata_CTCL = sc.AnnData.concatenate(adata_CTCL2, adata_CTCL5, adata_CTCL6, 
                                   adata_CTCL8, adata_CTCL12, batch_key='sample', 
                                   batch_categories=['CTCL2', 'CTCL5', 'CTCL6',
                                                     'CTCL8', 'CTCL12'])
adata_HC = sc.AnnData.concatenate(adata_HC1, adata_HC2, adata_HC3, 
                                   adata_HC4, batch_key='sample', batch_categories=[
                                       'HC1', 'HC2', 'HC3', 'HC4'
                                   ])

In [ ]:
# Basic QC filtering
adata_HC.var['mt'] = adata_HC.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_HC, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(adata_HC, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(adata_HC, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_HC, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata_HC = adata_HC[((adata_HC.obs.n_genes_by_counts < 5500) & 
                                    (adata_HC.obs.n_genes_by_counts > 400)).values, :]
adata_HC = adata_HC[adata_HC.obs.pct_counts_mt < 30, :]

In [ ]:
sc.pp.filter_genes(adata_HC, min_counts=1)
sc.pp.log1p(adata_HC)
sc.pp.normalize_total(adata_HC)

In [ ]:
sc.pp.pca(adata_HC, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_HC, metric='angular', batch_key='sample')
tk.tl.triku(adata_HC, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_HC, min_dist=0.1, random_state=seed)
sc.tl.leiden(adata_HC, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_HC, color=['leiden', 'sample'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_HC, color=['leiden', 'LUM', 'PDGFRA', 'COL1A1', 'DCN'], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
adata_HC_fb = adata_HC[adata_HC.obs['leiden'].isin(['0', '1', '3', '8', '9', '31'])]

In [ ]:
sc.pp.filter_genes(adata_HC_fb, min_counts=1)
sc.pp.pca(adata_HC_fb, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_HC_fb, metric='angular', batch_key='sample')
tk.tl.triku(adata_HC_fb, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_HC_fb, min_dist=0.1, random_state=seed)
sc.tl.leiden(adata_HC_fb, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_HC_fb, color=['leiden'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_HC_fb, color=['leiden', 'COL18A1', 'COMP', 'APCDD1', 'SLPI', 'WIF1'], legend_loc='on data', ncols=2, cmap=magma)

In [ ]:
sc.pl.umap(adata_HC_fb, color=['leiden', 'CCL19', 'CD74', 'APOE', 
                              ], legend_loc='on data', ncols=2, cmap=magma)

In [ ]:
sc.pl.umap(adata_HC_fb, color=['leiden', 'COL11A1', 'DPEP1', 'COCH', 'CRABP1', 
                               'ASPN', 'POSTN', 'ANGPTL7', 'C2orf40'], legend_loc='on data', ncols=2, cmap=magma)